In [142]:
import pandas as pd
import numpy as np
import pickle as p

In [143]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=153, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [144]:
# item_subset

### `/train`

In [145]:
import time

In [146]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'
# url = 'http://localhost:5000/train/'


data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 1}

Thread(target=requests.post, args=(url, ), kwargs={'data': payload, 'files': files}).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [154]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id': 1}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 100.0
CV % Complete: 32.0
CV Done: 0


In [110]:
set(r.json()['project_items']) - set(r.json()['train_complete'])

set()

### `/get_cv_results`

In [153]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': 1}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,1002.0,2610.850342,0.649976,653.276560,0.250216,788.212616
1,1003.0,3376.867676,0.918006,276.437648,0.081862,412.883962
2,1004.0,3464.819580,0.889222,349.529195,0.100879,462.898905
3,101.0,210.199631,0.801426,28.869619,0.137344,36.880814
4,102.0,178.524384,0.688875,31.438341,0.176101,47.072151
5,1152.0,113.752693,0.594906,23.412595,0.205820,29.448702
6,1153.0,281.756165,0.825182,37.380898,0.132671,49.769918
7,1195.0,130.380447,0.710220,24.445138,0.187491,30.499925
8,11.0,218.526474,0.870352,26.046635,0.119192,35.559450
9,1203.0,199.803940,0.858945,23.969528,0.119965,33.569090


In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [60]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
# a_input


# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'
# url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

Input Prices Quantities:


,Estimated,Actual
Qty_1004,4010.0,3189.0
Qty_102,249.0,183.0
Qty_11,273.0,229.0
Qty_1152,115.0,101.0
Qty_1195,129.0,119.0
...,...,...
Qty_7434,206.0,148.0
Qty_7575,107.0,69.0
Qty_7965,46.0,51.0
Qty_7966,146.0,160.0
